# y-y' emittance

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
import importlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
import h5py
from scipy import ndimage
import proplot as pplt

sys.path.append('..')
from tools import image_processing as ip
from tools import utils
from tools import plotting as mplt
from tools import energyVS06 as energy

sys.path.append('/Users/46h/Research/btf/btf-scripts/')
import scan_patterns as sp

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

In [ ]:
datadir = '../Diagnostics/Data/Measurements/2022-06-16/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220616114247-y-emittance2d.h5'
file = h5py.File(join(datadir, filename), 'r')
data = file['scandata']
pprint(data.dtype.fields)

In [ ]:
acts = ['y_PositionSync', 'yp_PositionSync']
signame = 'cam34_Integral'
sdiag = ['cam34_Integral', 'cam34_Saturation', 'bcm04']
signal = data[signame]

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
cam = 'cam34'
cam_settings = ip.camera_settings(cam)

In [ ]:
fig, ax = pplt.subplots(figsize=(8, 2))
for act in acts:
    ax.plot(data[:, act], marker=None, label=act)  # plt.plot(tpred,act3,':',color='C2')
ax.legend()
ax.format(xlabel='Step', ylabel='[mm]')

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[1]], data[:, acts[0]], marker='s', s=50, 
           c=signal, norm='log', 
           colorbar=True, colorbar_kw=dict(label=signame))
ax.format(xlabel=acts[1], ylabel=acts[0])
# plt.savefig('_output/y2d')

In [ ]:
for item in sdiag:
    print(f'Diagnostic: {item}')
    print('Max = {:.3f}'.format(np.max(data[:, item])))
    print('Min = {:.3f}'.format(np.min(data[:, item])))
    print('Mean = {:.3f}'.format(np.mean(data[:, item])))
    fig, ax = pplt.subplots(figsize=(7.0, 2.0))
    ax.plot(data[:, item], color='black', marker='.', ms=4, label=item)
    ax.format(xlabel='Step', ylabel=item)
    plt.show()

#### Pick frames for background calculation

In [ ]:
bgidx = np.arange(1, 35)

fig, ax = pplt.subplots()
ax.plot(data[:, acts[0]], data[:, acts[1]], alpha=0.2, color='grey')
ax.scatter(data[:, acts[0]], data[:, acts[1]], marker='s', 
           c=signal, norm='log',
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))

ax.plot(data[bgidx, acts[0]], data[bgidx, acts[1]], 'rs', label='Background')
ax.legend(loc='t')
ax.format(xlabel=acts[0], ylabel=acts[1])

## Threshold 

In [ ]:
# thr = 0.043
thr = 0.1075
w = signal.copy()
w[w < thr] = 0.0
print(f'signame = {signame}')
print('Est 4D dynamic range 10^%.3f'%np.log10(thr / max(w)))

In [ ]:
idx, = np.where(signal >= thr)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(signal, color='black', lw=0, marker='.', s=5)
ax.semilogy(idx, signal[idx], marker='.', s=1, color='red', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(np.sort(signal), color='black', lw=0, marker='.', s=5)
ax.semilogy(np.sort(w), marker='.', color='red', lw=0, s=1, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame,)

In [ ]:
fig, axes = pplt.subplots(ncols=2)
for mask, ax in zip([False, True], axes):
    c = np.ma.masked_less_equal(signal, thr) if mask else signal
    ax.scatter(
        data[:, acts[1]], data[:, acts[0]], marker='s', s=50,
        c=c, norm='log',
        colorbar=True
    )
axes[1].format(xlim=axes[0].get_xlim(), ylim=axes[0].get_ylim())
axes.format(xlabel=acts[1], ylabel=acts[0])

## Measure against new scan boundaries

Make a list of coordinates that have signal.

In [ ]:
pts = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
sig_pts = pts[idx, :]

Define "normalized" actuator coordinates.

In [ ]:
ndim = 2
nsteps = 32
M = np.identity(ndim)
M[0, 1] = 0.55
slit_dict = {
    'y1': {
        'pvname': 'ITSF_Diag:Slit_VT34b',
        'center': 6.5,
        'distance': 16.5,
        'steps': nsteps,
        'min': -50.0,
        'max': +50.0,
    },
    'y2': {
        'pvname': 'ITSF_Diag:Slit_VT34a',
        'center': 9.5,
        'distance': 18.0,
        'steps': nsteps,
        'min': -50.0, 
        'max': +50.0
    }
}
reprate = 5
navg = 0
keys = ['y1', 'y2']
for key in keys:
    slit_dict[key]['distance'] *= 1.0

In [ ]:
importlib.reload(sp)

In [ ]:
def gen(navg=1): 
    center = np.array([slit_dict[keys[i]]['center'] for i in range(ndim)])
    distance = np.array([slit_dict[keys[i]]['distance'] for i in range(ndim)])
    nsteps = np.array([slit_dict[keys[i]]['steps'] for i in range(ndim)])
    
    grid = sp.ellipsoid(center, distance, nsteps, fit=navg==0)
        
    ##### STEPPER SCAN COMMANDS
    if navg > 0:           
        for point in grid:
            realpoint = np.matmul(M, point - center) + center
            # don't let slit destination exceed min/max specified in slit_dict
            # skip points that exceed this limit, but otherwise do not change scan
            skip = False
            for i in range(ndim):
                if (realpoint[i] > slit_dict[keys[i]]['max']) or (realpoint[i] < slit_dict[keys[i]]['min']):
                    skip = True
            if skip:
                continue
            yield realpoint, navg
        
    ##### SWEEPER SCAN COMMANDS
    elif navg == 0:
        sweep_speed = distance[0]/nsteps[0]*reprate
        nsteps[0] = 1

        # -- call grid instance and get first point (note that this point is
        # now removed from `grid`)
        lastpoint = next(grid)
        # -- step through grid points
        for point in grid:
            # -- skip points where sweeper doesn't move
            if (point[0] - lastpoint[0]) == 0:
                lastpoint = point.copy()
                continue
                
            # -- transform to real slit coordinates
            realpoint = np.matmul(M, point - center) + center
            reallastpoint = np.matmul(M, lastpoint - center) + center
            
            # -- copy last point for next iteration
            lastpoint = point.copy()
            
            # -- parse moving/stationary slits on this step
            istay, imove = [], []
            for j in range(ndim):
                dxj = np.abs(realpoint[j] - reallastpoint[j])
                if dxj == 0:  # this point is a stepping point
                    istay.append(j)
                else:
                    imove.append(j)
                 
            ## - this block applies conditions for corner-cutting
            # if slit isn't moving but is in invalid region, skip this step
            skip = False
            for i in istay:
                if (realpoint[i] > slit_dict[keys[i]]['max']) or (realpoint[i] < slit_dict[keys[i]]['min']):
                    skip = True
            if skip:
                continue
            # constrain sweepers to valid region. This may cause
            # odd behavior if there is > 1 sweeper.
            for i in imove:
                if (reallastpoint[i] > slit_dict[keys[i]]['max']):
                    reallastpoint[i] = slit_dict[keys[i]]['max']
                if (reallastpoint[i] < slit_dict[keys[i]]['min']):
                    reallastpoint[i] = slit_dict[keys[i]]['min']
                if (realpoint[i] > slit_dict[keys[i]]['max']):
                    realpoint[i] = slit_dict[keys[i]]['max']
                if (realpoint[i] < slit_dict[keys[i]]['min']):
                    realpoint[i] = slit_dict[keys[i]]['min']
            # -- don't send a command if nothing moves 
            # (happens on 1st, last pts)
            if np.all(np.array(reallastpoint) == np.array(realpoint)):
                continue
            # - send command if it passed all checks above
            # -- build up cmd list
            cmd = []
            # -- 1st slit always gets sweeper command
            cmd.append((reallastpoint[0], realpoint[0], sweep_speed))
            # -- commands for the rest of the actuators
            for j in range(1, ndim):
                # if slope, stepper may have to sweep as well
                stepper_speed = sweep_speed * (np.abs(realpoint[j] - reallastpoint[j]) / distance[0])
                cmd.append((reallastpoint[j], realpoint[j], stepper_speed))
            yield cmd

In [ ]:
# Run points generator
lgen = list(gen(navg=navg))

# Reshape
if navg > 0:
    new_pts = np.zeros((len(lgen), ndim))
    for i in range(len(lgen)):
        new_pts[i, :] = lgen[i][0]
else:
    lgen = np.array(lgen)
    new_pts = np.zeros((2 * lgen.shape[0], ndim))
    for i in range(ndim):
        new_pts[:, i] = lgen[:, i, :2].ravel()
    
# Un-shear new (generated) points.
center = np.array([slit_dict['y1']['center'], slit_dict['y2']['center']])
Minv = np.linalg.inv(M)
new_pts_n = utils.apply(Minv, new_pts - center) + center
new_pts_n = utils.apply(Minv, new_pts - center) + center

# Un-shear signal points.
sig_pts_n = np.zeros(sig_pts.shape)
for row in range(sig_pts.shape[0]):
    sig_pts_n[row, :] = np.matmul(Minv, sig_pts[row, :] - center) + center
    
# Get actuator coordinates.
coords = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
coords_n = np.zeros(coords.shape)
for row in range(coords.shape[0]):
    coords_n[row, :] = np.matmul(Minv, coords[row, :] - center) + center
    
# Compute ellipse coordinates.
center = np.array([slit_dict[keys[i]]['center'] for i in range(ndim)])
distance = np.array([slit_dict[keys[i]]['distance'] for i in range(ndim)])
phi = np.linspace(0.0, 2.0 * np.pi, 100)
xx_n = 0.5 * distance[0] * np.cos(phi) 
yy_n = 0.5 * distance[1] * np.sin(phi)
xx, yy = utils.apply(M, np.vstack([xx_n, yy_n]).T).T

In [ ]:
fig, axes = pplt.subplots(ncols=2, wspace=None, figwidth=None)
c = np.ma.masked_less_equal(signal, thr) if mask else signal
for ax, _pts, _new_pts in zip(axes, [sig_pts, sig_pts_n], [new_pts, new_pts_n]):
    ax.scatter(_pts[:, 0], _pts[:, 1], marker='s', color='pink1', label='signal')
    ax.plot(_new_pts[:, 0], _new_pts[:, 1], marker='.', s=3, lw=1.0, color='grey', label='scan')
axes.format(xlabel=acts[0], ylabel=acts[1], toplabels=['Sheared', 'Un-sheared'])
for ax, x, y in zip(axes, [xx, xx_n], [yy, yy_n]):
    ax.plot(x + center[0], y + center[1], color='black', ls='-', lw=1.0)
for ax in axes:
    xmin, xmax = ax.get_xlim()
    if xmin > xmax:
        xmin, xmax = xmax, xmin
    ax.format(xlim=(xmin, xmax))
for ax in axes:
    xmin, xmax = ax.get_xlim()
    width = np.abs(xmax - xmin)
    xmid = 0.5 * (xmax + xmin)
    pad = 0.1
    xmin = xmid - (0.5 + pad) * width
    xmax = xmid + (0.5 + pad) * width
    ax.format(xlim=(xmin, xmax))
axes[0].legend(ncols=1, loc='upper left')
# plt.savefig(f'yyp_ellipse_scan_navg{navg}.png')
plt.show()

## Convert to phase space coordinates 

In [ ]:
y1 = data['y_PositionSync'].copy()
y2 = data['yp_PositionSync'].copy()

In [ ]:
a2mm = 1.009  # assume same as first dipole
rho = 0.3556
GL05 = 0.0
GL06 = 0.0
l1 = 0.0
l2 = 0.0
l3 = 0.774
L2 = 0.311  # slit2 to dipole face
l = 0.129  # dipole face to VS06 screen (assume same for first/last dipole-screen)
LL = l1 + l2 + l3 + L2  # distance from emittance plane to dipole entrance

ecalc = energy.EnergyCalculate(l1=l1, l2=l2, l3=l3, L2=L2, l=l, amp2meter=a2mm*1e3)
Mslit = ecalc.getM1()  # slit-slit
Mscreen = ecalc.getM()  # slit-screen

# Watch the negative signs!
y = y1
yp = 1e3 * ecalc.calculate_yp(y1 * 1e-3, y2 * 1e-3, Mslit)  
y = y - np.mean(y)
yp = yp - np.mean(yp)

In [ ]:
points = np.vstack([y, yp]).T
values = signal

n = 28
y_gv = np.linspace(np.min(y), np.max(y), n)
yp_gv = np.linspace(np.min(yp), np.max(yp), int(1.4 * n))
Y, YP = np.meshgrid(y_gv, yp_gv, indexing='ij')
new_points = np.vstack([Y.ravel(), YP.ravel()]).T

from scipy import interpolate
f = interpolate.griddata(points, values, new_points, method='linear', fill_value=0.0)
f = f.reshape(len(y_gv), len(yp_gv))

fig, axes = pplt.subplots(ncols=2)
axes[0].scatter(y, yp, c=signal, marker='.')
for g in y_gv:
    axes[0].axvline(g, color='black', alpha=0.1)
for g in yp_gv:
    axes[0].axhline(g, color='black', alpha=0.1)
f = np.clip(f - 0.03 * f.max(), 0.0, None)
mplt.plot_image(f / np.max(f), x=y_gv, y=yp_gv, ax=axes[1], colorbar=True)
axes.format(xlabel='y [mm]', ylabel='yp [mrad]')